In [134]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [135]:
data = pd.read_csv('data.csv')
data.set_index('shot_id', inplace=True)

In [136]:
necessaryDatas = data.copy()
target = necessaryDatas['shot_made_flag'].copy()
necessaryDatas.drop('team_id', inplace=True, axis=1)
necessaryDatas.drop('lat', inplace=True, axis=1) 
necessaryDatas.drop('lon', inplace=True, axis=1) 
necessaryDatas.drop('game_id', inplace=True, axis=1)
necessaryDatas.drop('game_event_id', inplace=True, axis=1)
necessaryDatas.drop('matchup', axis=1, inplace=True)
necessaryDatas.drop('team_name', inplace=True, axis=1)
necessaryDatas.drop('shot_made_flag', inplace=True, axis=1) 
necessaryDatas.drop('minutes_remaining', axis=1, inplace=True)
necessaryDatas.drop('seconds_remaining', axis=1, inplace=True)

In [137]:
necessaryDatas['game_date'] = pd.to_datetime(necessaryDatas['game_date'])
necessaryDatas['game_year'] = necessaryDatas['game_date'].dt.year
necessaryDatas['game_month'] = necessaryDatas['game_date'].dt.month
necessaryDatas.drop('game_date', axis=1, inplace=True)
rare_action_types = necessaryDatas['action_type'].value_counts().sort_values().index.values[:20]
necessaryDatas.loc[necessaryDatas['action_type'].isin(rare_action_types), 'action_type'] = 'Other'

In [138]:
categorialColums = [
    'action_type', 'combined_shot_type', 'period', 'season', 'shot_type',
    'shot_zone_area', 'shot_zone_basic', 'shot_zone_range', 'game_year',
    'game_month', 'opponent']
for i in categorialColums:
    dummies = pd.get_dummies(necessaryDatas[i])
    dummies = dummies.add_prefix("{}_".format(i))
    necessaryDatas.drop(i, axis=1, inplace=True)
    necessaryDatas = necessaryDatas.join(dummies)

In [139]:
unknownShot = data['shot_made_flag'].isnull()
data_submit = necessaryDatas[unknownShot]
X = necessaryDatas[~unknownShot]
Y = target[~unknownShot]
val_mask = np.random.rand(len(X)) < 0.7 
X_train = X[val_mask]
Y_train = Y[val_mask]
X_val = X[~val_mask]
Y_val = Y[~val_mask]

In [140]:
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=161)]
dnnc = tf.contrib.learn.DNNClassifier(
  feature_columns=feature_columns,
  hidden_units=[50, 50, 50, 25],
  n_classes=2)

In [141]:
def print_accuracy():
  loss = dnnc.evaluate(x=X_val, y=Y_val)['loss']
  print('Loss: {:.2%}'.format(loss))  
dnnc.fit(x=X_train, y=Y_train, steps=1)
print_accuracy()

Loss: 545.11%


In [142]:
steps = 500
for i in range (1, 6):
  dnnc.fit(x=X_train, y=Y_train, steps=steps)
  print('Steps: ' + str(i * steps))
  print_accuracy()

Steps: 500
Loss: 66.55%
Steps: 1000
Loss: 65.97%
Steps: 1500
Loss: 64.28%
Steps: 2000
Loss: 63.73%
Steps: 2500
Loss: 62.54%


In [143]:
preds = dnnc.predict_proba(data_submit, as_iterable=False)
prediction = pd.DataFrame()
prediction["shot_id"] = data_submit.index
result = []
temp = preds[:,1]
for i in range(len(temp)):
    if temp[i] <= 0.5:
        result.append(0)
    else:
        result.append(1)
prediction["shot_made_flag"] = result
prediction["Prediction"]= preds[:,1]
prediction.to_csv("prediction.csv",index=False)